In [1]:
import numpy as np
import pandas as pd
import timer
%load_ext cython

In [2]:
from model_breakdown_validation import route_model

In [3]:
temperature_ranges = {
    "< -30C":-30,
    "-29C to -25C":-27,
    "-24C to -20C":-22,
    "-19C to -15C":-17,
    "-14C to -10C":-12,
    "-9C to -5C":-7,
    "-4C to 0C":-2,
    "0C to 4C":2,
    "5C to 9C":7,
    "10C to 14C":12,
    "15C to 19C":17,
    "20C to 24C":22,
    "25C to 29C":27,
    "30C to 34C":32,
    "35C to 39C":37,
    "> 40C":40,
}
initial_values = {
    'template_name': 'Default Battery Template',
    'GRAVITY': 9.81,
    'RHO_AIR': 1.225,
    'gvw': 18327,
    'wheel_crr': 0.007,
    'frontal_area' : 8.783466304,
    'vehicle_cd' : 0.8,
    'pwr_aux_base': 8,
    'pwr_aux_pcnt': 0,
    'pwr_conv_eff': 0.95,
    'trans_eff': 0.98,
    'battery_engine': "be",
    'ic_engine': ["ic", "ic_h"],
    'fc_engine': ["fc", "fc_h"],
    'hybrid': ["ic_h", "fc_h"],
    'engine_type': 'be',
    'ic_pwr_max': 0,
    'ic_time_full' : 0,
    'ic_time_zero' : 0,
    'emotor_max_eff' : 0.95,
    'emotor_pwr_max' : 380,
    'emotor_time_full' : 4,
    'fc_pwr_max' : 0,
    'fc_time_full' : 0,
    'fc_time_zero' : 0,
    'regen_a' : 500,
    'regen_b' : 0.99,
    'regen_eff_max' : 0.98,
    'max_regen_pwr' : 33,
    'pwr_conv_eff' : 0.95,
    'ess_pwr_max'  : 380,
    'ess_round_trip_eff' : 0.97,
    'ess_time_full' : 1,
    'ess_cap' : 594,
    'ess_init_soc' : 0.95,
    'ess_target_soc' : 1,
    'ess_min_soc' : 0.10,
    'ess_max_soc' : 0.95,
    'ess_soc_gain' : 0.3,
    'h2_lhv' : 33.32,
}

In [4]:
df = pd.read_csv('./raw_data/block_db_lists/roam_elev.csv')
df.drop(columns=['Driver ID', 'Route ID', 'Heading', 'Stop ID'], inplace=True)
df['filename'] = df['Bus ID'] + " " + df['Date']
df['Speed'] = df['Speed'] * 3.6
df.rename(columns={'SAMPLE_1':'Elevation'}, inplace=True)
df['DateTime'] = df['Time']
df['DateTime'] = pd.to_datetime(df['DateTime'])
df.head()

,Bus ID,Latitude,Longitude,Speed,Time,filename,Date,Elevation,DateTime
0,1033_SummerWolf,51.19189,-115.55418,5.076,2022/03/15 05:42:46,1033_SummerWolf 2022/03/15,2022/03/15,1401,2022-03-15 05:42:46
1,1033_SummerWolf,51.19189,-115.55420,5.292,2022/03/15 05:42:47,1033_SummerWolf 2022/03/15,2022/03/15,1401,2022-03-15 05:42:47
2,1033_SummerWolf,51.19187,-115.55427,5.148,2022/03/15 05:42:49,1033_SummerWolf 2022/03/15,2022/03/15,1401,2022-03-15 05:42:49
3,1033_SummerWolf,51.19187,-115.55430,3.132,2022/03/15 05:42:50,1033_SummerWolf 2022/03/15,2022/03/15,1401,2022-03-15 05:42:50
4,1033_SummerWolf,51.19187,-115.55431,2.448,2022/03/15 05:42:51,1033_SummerWolf 2022/03/15,2022/03/15,1401,2022-03-15 05:42:51


In [5]:
blocks = df['filename'].unique().tolist()

df_dict = {block: df.loc[df['filename'] == block] for block in blocks}
df_dict

{'1033_SummerWolf 2022/03/15':                 Bus ID  Latitude  Longitude  Speed                 Time  \
 0      1033_SummerWolf  51.19189 -115.55418  5.076  2022/03/15 05:42:46   
 1      1033_SummerWolf  51.19189 -115.55420  5.292  2022/03/15 05:42:47   
 2      1033_SummerWolf  51.19187 -115.55427  5.148  2022/03/15 05:42:49   
 3      1033_SummerWolf  51.19187 -115.55430  3.132  2022/03/15 05:42:50   
 4      1033_SummerWolf  51.19187 -115.55431  2.448  2022/03/15 05:42:51   
 ...                ...       ...        ...    ...                  ...   
 33283  1033_SummerWolf  51.19060 -115.56129  0.036  2022/03/15 23:59:55   
 33284  1033_SummerWolf  51.19060 -115.56129  0.036  2022/03/15 23:59:57   
 33285  1033_SummerWolf  51.19060 -115.56129  0.036  2022/03/15 23:59:58   
 33286  1033_SummerWolf  51.19060 -115.56129  0.036  2022/03/15 23:59:59   
 33287  1033_SummerWolf  51.19060 -115.56129  0.036  2022/03/16 00:00:00   
 
                          filename        Date  Elevatio

In [6]:
for key, df in df_dict.items():
    df.reset_index(inplace=True, drop=True)
    init_time = pd.to_datetime(df.loc[0, 'Time'])
    df['Time'] = ((df['DateTime']) - init_time) / pd.Timedelta(seconds=1)

df_dict['1030_Hare 2022/03/15']

,Bus ID,Latitude,Longitude,Speed,Time,filename,Date,Elevation,DateTime
0,1030_Hare,51.19143,-115.55373,0.972,0.0,1030_Hare 2022/03/15,2022/03/15,1401,2022-03-15 06:02:31
1,1030_Hare,51.19139,-115.55367,0.396,1.0,1030_Hare 2022/03/15,2022/03/15,1401,2022-03-15 06:02:32
2,1030_Hare,51.19137,-115.55368,0.684,2.0,1030_Hare 2022/03/15,2022/03/15,1401,2022-03-15 06:02:33
3,1030_Hare,51.19135,-115.55368,1.584,4.0,1030_Hare 2022/03/15,2022/03/15,1401,2022-03-15 06:02:35
4,1030_Hare,51.19137,-115.55372,1.044,5.0,1030_Hare 2022/03/15,2022/03/15,1401,2022-03-15 06:02:36
...,...,...,...,...,...,...,...,...,...
2189,1030_Hare,51.17955,-115.56992,0.036,2653.0,1030_Hare 2022/03/15,2022/03/15,1402,2022-03-15 06:46:44
2190,1030_Hare,51.17956,-115.56992,0.072,2654.0,1030_Hare 2022/03/15,2022/03/15,1402,2022-03-15 06:46:45
2191,1030_Hare,51.17956,-115.56993,0.036,2656.0,1030_Hare 2022/03/15,2022/03/15,1402,2022-03-15 06:46:47
2192,1030_Hare,51.17956,-115.56993,0.036,2657.0,1030_Hare 2022/03/15,2022/03/15,1402,2022-03-15 06:46:48


In [2]:
%%cython -a
import numpy as np
import pandas as pd
from model_breakdown_validation import route_model

temperature_ranges = {
    "< -30C":-30,
    "-29C to -25C":-27,
    "-24C to -20C":-22,
    "-19C to -15C":-17,
    "-14C to -10C":-12,
    "-9C to -5C":-7,
    "-4C to 0C":-2,
    "0C to 4C":2,
    "5C to 9C":7,
    "10C to 14C":12,
    "15C to 19C":17,
    "20C to 24C":22,
    "25C to 29C":27,
    "30C to 34C":32,
    "35C to 39C":37,
    "> 40C":40,
}

initial_values = {
    'template_name': 'Default Battery Template',
    'GRAVITY': 9.81,
    'RHO_AIR': 1.225,
    'gvw': 18327,
    'wheel_crr': 0.007,
    'frontal_area' : 8.783466304,
    'vehicle_cd' : 0.8,
    'pwr_aux_base': 8,
    'pwr_aux_pcnt': 0,
    'pwr_conv_eff': 0.95,
    'trans_eff': 0.98,
    'battery_engine': "be",
    'ic_engine': ["ic", "ic_h"],
    'fc_engine': ["fc", "fc_h"],
    'hybrid': ["ic_h", "fc_h"],
    'engine_type': 'be',
    'ic_pwr_max': 0,
    'ic_time_full' : 0,
    'ic_time_zero' : 0,
    'emotor_max_eff' : 0.95,
    'emotor_pwr_max' : 380,
    'emotor_time_full' : 4,
    'fc_pwr_max' : 0,
    'fc_time_full' : 0,
    'fc_time_zero' : 0,
    'regen_a' : 500,
    'regen_b' : 0.99,
    'regen_eff_max' : 0.98,
    'max_regen_pwr' : 33,
    'pwr_conv_eff' : 0.95,
    'ess_pwr_max'  : 380,
    'ess_round_trip_eff' : 0.97,
    'ess_time_full' : 1,
    'ess_cap' : 594,
    'ess_init_soc' : 0.95,
    'ess_target_soc' : 1,
    'ess_min_soc' : 0.10,
    'ess_max_soc' : 0.95,
    'ess_soc_gain' : 0.3,
    'h2_lhv' : 33.32,
}
##redefine df
df = pd.read_csv('./raw_data/block_db_lists/roam_elev.csv')
df.drop(columns=['Driver ID', 'Route ID', 'Heading', 'Stop ID'], inplace=True)
df['filename'] = df['Bus ID'] + " " + df['Date']
df['Speed'] = df['Speed'] * 3.6
df.rename(columns={'SAMPLE_1':'Elevation'}, inplace=True)
df['DateTime'] = df['Time']
df['DateTime'] = pd.to_datetime(df['DateTime'])
### df_dict
blocks = df['filename'].unique().tolist()
df_dict = {block: df.loc[df['filename'] == block] for block in blocks}
for key, df in df_dict.items():
    df.reset_index(inplace=True, drop=True)
    init_time = pd.to_datetime(df.loc[0, 'Time'])
    df['Time'] = ((df['DateTime']) - init_time) / pd.Timedelta(seconds=1)

temperature_ranges = temperature_ranges
temperature = temperature_ranges['20C to 24C']
duty_cycle_name = df_dict['1033_SummerWolf 2022/05/11']['Bus ID'][0]
duty_cycle = df_dict['1033_SummerWolf 2022/05/11']
duty_cycle.rename(columns={"Time":'time', 'Speed':'speed', 'Elevation':'elevation'}, inplace=True)

dc, results, results_dt, sum_table, distance, duration, mean_grade, max_grade, ave_speed, complete, \
           distance_complete, fuel_consumed, fuel_economy, engine_type = route_model(initial_values, duty_cycle, duty_cycle_name, temperature)

Time for variable assignment is 0.01s
Time for hvac calcs is 0.02s
Time for dc_forces calcs is 0.18s
Time for dc_power calcs is 0.01s
Time for aux_trans_power calcs is 0.01s
Time for engine_priorities calcs is 0.00s
Time for ice_power_limit calcs is 0.27s
Time for emotor_requirements calcs is 0.15s
BE Selected
Time for fuel_cell_init_beb calcs is 0.16s
Time for ess_requirements_beb calcs is 119.27s
Time for fuel consumption calcs is 0.10s
Time for results calcs is 0.76


## Numpy List Test

In [2]:
%load_ext cython
import numpy as np
import pandas as pd
from model_breakdown_validation import route_model

regen_map = pd.read_csv("./maps/regen_map.csv")
emotor_eff_map = pd.read_csv("./maps/emotor_eff_map.csv")

temperature_ranges = {
    "< -30C":-30,
    "-29C to -25C":-27,
    "-24C to -20C":-22,
    "-19C to -15C":-17,
    "-14C to -10C":-12,
    "-9C to -5C":-7,
    "-4C to 0C":-2,
    "0C to 4C":2,
    "5C to 9C":7,
    "10C to 14C":12,
    "15C to 19C":17,
    "20C to 24C":22,
    "25C to 29C":27,
    "30C to 34C":32,
    "35C to 39C":37,
    "> 40C":40,
}

model_input = {
    'template_name': 'Default Battery Template',
    'GRAVITY': 9.81,
    'RHO_AIR': 1.225,
    'gvw': 18327,
    'wheel_crr': 0.007,
    'frontal_area' : 8.783466304,
    'vehicle_cd' : 0.8,
    'pwr_aux_base': 8,
    'pwr_aux_pcnt': 0,
    'pwr_conv_eff': 0.95,
    'trans_eff': 0.98,
    'battery_engine': "be",
    'ic_engine': ["ic", "ic_h"],
    'fc_engine': ["fc", "fc_h"],
    'hybrid': ["ic_h", "fc_h"],
    'engine_type': 'be',
    'ic_pwr_max': 0,
    'ic_time_full' : 0,
    'ic_time_zero' : 0,
    'emotor_max_eff' : 0.95,
    'emotor_pwr_max' : 380,
    'emotor_time_full' : 4,
    'fc_pwr_max' : 0,
    'fc_time_full' : 0,
    'fc_time_zero' : 0,
    'regen_a' : 500,
    'regen_b' : 0.99,
    'regen_eff_max' : 0.98,
    'max_regen_pwr' : 33,
    'pwr_conv_eff' : 0.95,
    'ess_pwr_max'  : 380,
    'ess_round_trip_eff' : 0.97,
    'ess_time_full' : 1,
    'ess_cap' : 594,
    'ess_init_soc' : 0.95,
    'ess_target_soc' : 1,
    'ess_min_soc' : 0.10,
    'ess_max_soc' : 0.95,
    'ess_soc_gain' : 0.3,
    'h2_lhv' : 33.32,
}

# USER INPUT & SELECTED CONSTANTS
# ### Scientific Constants
GRAVITY = 9.81  # gravitational constant
RHO_AIR = 1.225

# ### Force User Inputs and Constants
gvw = model_input['gvw']  # 18327 #gross vehicle weight
wheel_crr = model_input['wheel_crr']  # 0.007 #Wheel rolling resistance coefficient
frontal_area = model_input['frontal_area']  # 8.783466304
vehicle_cd = model_input['vehicle_cd']  # 0.8 #Drag coefficient

# ### Auxilliary Power User Inputs and Constants
pwr_aux_base = model_input['pwr_aux_base']  # 8
pwr_aux_pcnt = model_input['pwr_aux_pcnt']  # 0
pwr_conv_eff = model_input['pwr_conv_eff']  # 0.95

# ### Transmission Power User Inputs and Constants
trans_eff = model_input['trans_eff']  # 0.98

# ### Drive Train Priorities and Engine Selection
# backend reference values
battery_engine = "be"
ic_engine = ["ic", "ic_h"]
fc_engine = ["fc", "fc_h"]
hybrid = ["ic_h", "fc_h"]

# Engine type for model
engine_type = model_input['engine_type']
ENGINE_SELECT = ""

# ### IC Engine User Inputs and Constants
ic_pwr_max = model_input['ic_pwr_max']  # 0
ic_time_full = model_input['ic_time_full']  # 0
ic_time_zero = model_input['ic_time_zero']  # 0

# ### Electrical Motor User Inputs and Constants
emotor_max_eff = model_input['emotor_max_eff']  # 0.95
emotor_pwr_max = model_input['emotor_pwr_max']  # 380
emotor_time_full = model_input['emotor_time_full']  # 4

# ### Fuel Cell (FC) Engine User Inputs and Constants
fc_pwr_max = model_input['fc_pwr_max']  # 0
fc_time_full = model_input['fc_time_full']  # 0
fc_time_zero = model_input['fc_time_zero']  # 0

# ### Regen Map Calculation Constants and User Inputs
regen_a = model_input['regen_a']  # 500
regen_b = model_input['regen_b']  # 0.99
regen_eff_max = model_input['regen_eff_max']  # 0.98 # Regen Breaking Efficiency
max_regen_pwr = model_input['max_regen_pwr']  # 33

# ### ESS User Inputs and Constants
# ESS Power Limits
pwr_conv_eff = model_input['pwr_conv_eff']  # 0.95 #( Power Converter Efficiency (%) )
ess_pwr_max = model_input['ess_pwr_max']  # 380  #( Energy Storage Power (kW) )
ess_round_trip_eff = model_input['ess_round_trip_eff']  # 0.97  #( Energy Storage Round Trip Efficiency (%) )
ess_time_full = model_input['ess_time_full']  # 1 #( Energy Storage Time to Full Power (s) )
ess_cap = model_input['ess_cap']  # 594 #( Energy Storage Capacity (kWh) )

# ESS State of Charge (SOC)
ess_init_soc = model_input['ess_init_soc']  # 0.95  #( ESS Initial State of charge (kWh) )
ess_target_soc = model_input['ess_target_soc']  # 1 #Hybrid Battery SOC target
ess_min_soc = model_input['ess_min_soc']  # 0.10 #( Battery min SOC (%) )
ess_max_soc = model_input['ess_max_soc']  # 0.95 #( Battery max SOC (%) )
ess_soc_gain = model_input['ess_soc_gain']  # 0.3 #SOC control proportional gain

# ### Fuel Type User Inputs and Constants
gas_lhv = 12.22  # Gasoline Heating Value (LHV)
diesel_lhv = 11.96  # Diesel Heating Value (LHV)
ng_lhv = 13.03  # Natural Gas Heating Value (LHV)
h2_lhv = 33.32  # Hydrogen Heating Value (LHV)
gas_density = 0.737  # Gasoline Density
diesel_density = 0.832  # Diesel Density
ng_density = 0.000733  # Natural Gas Density (STP)
h2_density = 0.000899  # Hydrogen Density
diesel_gge = 1.155  # GGE Per Gal Diesel
cng_gge = 0.00108  # GGE per Gal CNG @STP
h2_gge = 1.019  # Hydrogen kg per gallon of gasoline
elec_gge = 0.031  # GGE per kWh of electricity
FUEL_TANK_SIZE = 38  # kg

from np_ess_requirements import ess_requirements_beb
timer = Timer()
dc = pd.read_csv('./ess_req_test.csv')
timer.start()
dc = ess_requirements_beb(dc, ess_pwr_max, ess_init_soc, ess_cap,
                                  fc_pwr_max, pwr_conv_eff, fc_time_full, fc_time_zero,
                                  ess_round_trip_eff, ess_time_full, regen_eff_max, regen_a,
                                  regen_b, max_regen_pwr, ess_soc_gain, ess_target_soc, ess_min_soc, ess_max_soc)
timer.stop()

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [1]:
import time
class Timer:
    def __init__(self):
        self._start_time = None
        self.elapsed_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise f"Timer is not running. Use .start() to start it"

        self.elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None

In [ ]:
temperature = temperature_ranges['20C to 24C']

dc_list = []
dist_list = []
duration_list = []
mean_grade_list = []
max_grade_list = []
ave_speed_list = []
complete_list = []
distance_complete_list = []
fuel_consumed_list = []
fuel_economy_list = []
engine_type_list = []
model_time = []
number_rows = []

for key, duty_cycle in df_dict.items():
    t = Timer()
    duty_cycle_name = key
    duty_cycle.rename(columns={"Time":'time', 'Speed':'speed', 'Elevation':'elevation'}, inplace=True)
    t.start()
    print('{} model run starting' .format(duty_cycle_name))
    dc, results, results_dt, sum_table, distance, duration, mean_grade, max_grade, ave_speed, complete, \
           distance_complete, fuel_consumed, fuel_economy, engine_type = route_model(initial_values, duty_cycle, duty_cycle_name, temperature)
    t.stop()
    dc_list.append(duty_cycle_name)
    dist_list.append(distance)
    duration_list.append(duration)
    mean_grade_list.append(mean_grade)
    max_grade_list.append(max_grade)
    ave_speed_list.append(ave_speed)
    complete_list.append(complete)
    distance_complete_list.append(distance_complete)
    fuel_consumed_list.append(fuel_consumed)
    fuel_economy_list.append(fuel_economy)
    engine_type_list.append(engine_type)
    model_time.append(t.elapsed_time)
    number_rows.append(duty_cycle.shape[0])
    print('{} model completed in {} seconds' .format(duty_cycle_name,t.elapsed_time))



In [ ]:
results = pd.DataFrame({'Duty Cycle':dc_list, 'Distance':dist_list, 'Duration':duration_list, 'Mean Grade':mean_grade_list, 'Max Grade':max_grade_list,
                          'Average Speed':ave_speed_list, 'Complete %':complete_list, 'Distance Complete (km)':dist_list, 'Fuel Consumed':fuel_consumed_list,
                          'Fuel Economy':fuel_economy_list, 'Model Run Time':model_time, 'Size (Rows)':number_rows})
results

In [ ]:
results.to_excel('model_validation_notebook.xlsx')